## Базовые требования к домашкам

1. Формат - jupyter-тетрадка или скрипт на питоне
2. Мы запускаем ваши тетрадки с нуля, поэтому следите, чтобы не было 
- необъявленных переменных (удалили ячейку, а переменная продолжает использоваться)
- лишних принтов отладочной информации
3. Комментарии приветствуются!

## Задание
В рамках этого задания мы будем создавать программу, которая получая на вход отзыв, будет предсказывать, является отзыв положительным или отрицательным. Делать мы будем это таким образом: мы возьмём некоторое число заранее размеченных как положительные или отрицательные отзывов, выделим те слова, которые встречаются только в положительных или только в отрицательных отзывах, и будем считать, каких слов  в поступившем нам на проверку отзыве больше.


Мы будем работать по заранее определённому пайплайну:

1.  Сначала нам надо скачать дату -- соберите как минимум 60 (30 положительных  и 30 отрицательных) отзывов на похожие продукты (не надо мешать отзывы на отели с отзывами на ноутбуки) для составления "тонального словаря" (чем больше отзывов, тем лучше)  и 10 отзывов для проверки качества.   (2 балла в случае сбора путём парсинга, 1 - если найдете уже готовые данные или просто закопипастите без парсинга)

2. Токенизируйте слова,  приведите их к нижнему регистру и к начальной форме  (1 балл за токенизацию, 1 - за начальную форму)

3. Составьте 2 множества - в одном будут слова, которые встречаются только в положительных отзывах, а в другом - встречающиеся только в отрицательных. Попробуйте поиграть с частотностями и исключить шум (к примеру, выбросить слова, встречающиеся 1-2 раза) (2 балла) (если у вас получились пустые множества, уберите фильтр по частотности или увеличьте выборку)

4. Создайте функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и посчитайте качество при помощи accuracy (1  - за коректно работающую функцию, 1 - за подсчёт accuracy)

5. Предложите как минимум 2 способа улучшить эту программу с помощью добавления к ней любых мулек (1 балл за описание словами, 2 - если реализуете хотя бы один способ)

В случае, если после долгих мучений в п. 3 множества по объективным причинам не получается (покажите, что пытались) - отправляйте жабу - зачтём полный балл

In [160]:
import pandas as pd
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
from nltk import WordNetLemmatizer
from string import punctuation
from nltk.corpus import stopwords

#в работе использовались отзывы с сайта metacritic на фильмы "little women", "the godfather", "her" и "mad max: fury road"
#собирались отзывы вручную


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cheno\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\cheno\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [161]:
lemmatizer = WordNetLemmatizer()
sw = stopwords.words('english')


In [162]:
sw.extend(['mad', 'max', 'fury', 'road', 'little', 'woman', 'godfather', 'movie', 'film'])
sw


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [163]:
url = 'https://raw.githubusercontent.com/chenopodiumlang/nlp3rdyear/main/hw1/data.csv'
df = pd.read_csv(url, sep=';', names=["Rating", "Review"])
df
# файл лежит в репозитории рядом с этой тетрадкой


,Rating,Review
0,1,What an outstanding movie. I love the way that...
1,1,Another remake of Little Women--and this one i...
2,1,"Incredible writing, performances, cinematograp..."
3,1,This was definitely my favorite movie of the y...
4,1,"The best movie, the best screenplay،In one opi..."
...,...,...
65,1,Mad Max Fury Road is an awesome post apocalypt...
66,1,"An incredible, relentless, adrenaline fueled a..."
67,1,I'm going to be honest: this is the only Mad M...
68,1,Mad Max is back and better than ever. I really...


In [164]:
def my_tokenize_lemmatize(text):
    text = text.lower()  # приводим к нижнему регистру
    tokenized = word_tokenize(text)  # токенизируем
    tokens = []
    for word in tokenized:
        for x in punctuation:
            word = word.replace(x, '')
        tokens.append(word) #убираем пунктуацию
    res = ' '.join(lemmatizer.lemmatize(word)
                   for word in tokens if (word not in sw))  # убираем стоп-слова
    return res


In [165]:
df['Review'] = df['Review'].apply(lambda x: my_tokenize_lemmatize(x))
df


,Rating,Review
0,1,outstanding love way developed character als...
1,1,another remake woman one spectacular instead...
2,1,incredible writing performance cinematograph...
3,1,definitely favorite year even though expectat...
4,1,best best screenplay،in one opinion say comp...
...,...,...
65,1,awesome post apocalyptic first doubtful watch...
66,1,incredible relentless adrenaline fueled acti...
67,1,going honest seen know almost nothing one f...
68,1,back better ever really enjoy edge seat gre...


In [166]:
#разделяем выборку на трейн и тест:
df_train = df.iloc[:60,:]
df_test = df.iloc[60:, :]


In [167]:
positive_reviews = df_train[df_train["Rating"] == 1]
negative_reviews = df_train[df_train["Rating"] == 0]


In [168]:
#составляем словарь с частотами слов для позитивных отзывов
positive_reviews_text = ' '.join(positive_reviews['Review'])
positive_dictionary = {}
for word in positive_reviews_text.split():
    if word not in positive_dictionary:
        positive_dictionary[word] = 1
    else:
        positive_dictionary[word] = positive_dictionary[word] + 1
positive_dictionary


{'outstanding': 1,
 'love': 5,
 'way': 1,
 'developed': 1,
 'character': 4,
 'also': 2,
 'made': 10,
 'closeness': 1,
 'family': 3,
 'come': 1,
 'comingofage': 1,
 'close': 1,
 'child': 1,
 'grow': 1,
 'get': 4,
 'married': 1,
 'drift': 1,
 'life': 1,
 'another': 1,
 'remake': 1,
 'woman': 1,
 'one': 15,
 'spectacular': 5,
 'instead': 1,
 'telling': 1,
 'story': 5,
 'linear': 2,
 'fashion': 1,
 'gerwig': 4,
 'begin': 1,
 'present': 2,
 'flash': 2,
 'back': 2,
 'forward': 1,
 'etc': 1,
 'unpeeling': 1,
 'layer': 1,
 'meaning': 2,
 'emotion': 2,
 'process': 1,
 'cast': 3,
 'superb': 1,
 'saoirse': 3,
 'ronan': 3,
 'dazzling': 1,
 'camera': 1,
 'work': 3,
 'musical': 1,
 'score': 3,
 'would': 1,
 'choose': 1,
 'narrative': 1,
 'complexly': 1,
 'rather': 1,
 'straightforward': 1,
 'manner': 1,
 'latter': 1,
 'approach': 1,
 'already': 1,
 'done': 2,
 'ad': 1,
 'nauseum': 1,
 'secondly': 1,
 'jo': 3,
 'alcott': 1,
 'alter': 1,
 'ego': 1,
 'struggle': 1,
 'published': 1,
 'new': 4,
 'york': 

In [169]:
#аналогично для негативных отзывов
negative_reviews_text = ' '.join(negative_reviews['Review'])
negative_dictionary = {}
for word in negative_reviews_text.split():
    if word not in negative_dictionary:
        negative_dictionary[word] = 1
    else:
        negative_dictionary[word] = negative_dictionary[word] + 1
negative_dictionary


{'waste': 8,
 'time': 8,
 'money': 3,
 'fake': 3,
 'huge': 1,
 'many': 2,
 'good': 4,
 'actor': 4,
 'talent': 1,
 'well': 4,
 'confusing': 1,
 'disjointed': 1,
 'ca': 4,
 'nt': 13,
 'hold': 1,
 'candle': 1,
 'winona': 1,
 'ryder': 1,
 'version': 1,
 'interesting': 3,
 'thing': 4,
 'costume': 2,
 'acting': 5,
 'generally': 2,
 'amy': 1,
 'terrible': 5,
 'wow': 1,
 'happening': 3,
 'critic': 3,
 'day': 2,
 'awful': 3,
 'didnt': 1,
 'even': 9,
 'notice': 1,
 'taken': 1,
 'art': 1,
 'firstly': 1,
 'writing': 1,
 'incredibly': 2,
 'poor': 2,
 'dialogue': 2,
 'stilted': 1,
 'unnatural': 1,
 'likely': 1,
 'reflected': 1,
 'book': 1,
 'literally': 1,
 'adequately': 1,
 'adapted': 1,
 'screenplay': 2,
 'secondly': 1,
 'direction': 2,
 'major': 1,
 'gap': 1,
 'plot': 6,
 'making': 1,
 'seem': 2,
 'incoherent': 1,
 'left': 2,
 'viewer': 1,
 'disoriented': 1,
 'thirdly': 1,
 'fair': 1,
 'midling': 1,
 'lightweight': 1,
 'nothing': 4,
 'anchor': 1,
 'poorly': 1,
 'written': 1,
 'part': 1,
 'cinemat

In [170]:
sorted_positive_dictionary = dict(sorted(
    positive_dictionary.items(), key=lambda x: x[1], reverse=True))
sorted_positive_dictionary


{'best': 19,
 'one': 15,
 'action': 13,
 'made': 10,
 'ever': 10,
 'performance': 8,
 'masterpiece': 8,
 'time': 6,
 'amazing': 6,
 'movie': 6,
 'well': 6,
 'really': 6,
 'love': 5,
 'spectacular': 5,
 'story': 5,
 'cinematography': 5,
 'like': 5,
 'scene': 5,
 'acting': 5,
 'nt': 5,
 'seen': 5,
 'character': 4,
 'get': 4,
 'gerwig': 4,
 'new': 4,
 'year': 4,
 'say': 4,
 'soundtrack': 4,
 'much': 4,
 'family': 3,
 'cast': 3,
 'saoirse': 3,
 'ronan': 3,
 'work': 3,
 'score': 3,
 'jo': 3,
 'march': 3,
 'think': 3,
 'experience': 3,
 'watch': 3,
 'incredible': 3,
 'even': 3,
 'simply': 3,
 'film': 3,
 'everything': 3,
 'frame': 3,
 'last': 3,
 'every': 3,
 'perfection': 3,
 'give': 3,
 'greatest': 3,
 'direction': 3,
 'spike': 3,
 'jonze': 3,
 'script': 3,
 '2013': 3,
 'good': 3,
 'phoenix': 3,
 'far': 3,
 'gorgeous': 3,
 'breathtaking': 3,
 'feel': 3,
 'also': 2,
 'linear': 2,
 'present': 2,
 'flash': 2,
 'back': 2,
 'meaning': 2,
 'emotion': 2,
 'done': 2,
 'world': 2,
 'deserves': 2,
 

In [171]:
sorted_negative_dictionary = dict(sorted(
    negative_dictionary.items(), key=lambda x: x[1], reverse=True))
sorted_negative_dictionary


{'nt': 13,
 'boring': 10,
 'even': 9,
 'seen': 9,
 'waste': 8,
 'time': 8,
 'ever': 8,
 'like': 8,
 'character': 8,
 'hour': 7,
 'one': 7,
 'really': 7,
 'action': 7,
 'plot': 6,
 'year': 6,
 'acting': 5,
 'terrible': 5,
 'story': 5,
 'worst': 5,
 'see': 5,
 'good': 4,
 'actor': 4,
 'well': 4,
 'ca': 4,
 'thing': 4,
 'nothing': 4,
 'love': 4,
 'may': 4,
 'human': 4,
 'bad': 4,
 'watch': 4,
 'car': 4,
 'money': 3,
 'fake': 3,
 'interesting': 3,
 'happening': 3,
 'critic': 3,
 'awful': 3,
 'saw': 3,
 'pretty': 3,
 'feel': 3,
 'life': 3,
 'two': 3,
 'simply': 3,
 'chase': 3,
 'thought': 3,
 'theater': 3,
 'vision': 3,
 'absolutely': 3,
 'pointless': 3,
 'completely': 3,
 'positive': 3,
 'review': 3,
 'mind': 3,
 'know': 3,
 'movie': 3,
 'everyone': 3,
 'going': 3,
 'effect': 3,
 'last': 3,
 'original': 3,
 'first': 3,
 'anyone': 3,
 '90': 3,
 'ridiculous': 3,
 'development': 3,
 'minute': 3,
 'sub': 3,
 'many': 2,
 'costume': 2,
 'generally': 2,
 'day': 2,
 'incredibly': 2,
 'poor': 2,
 '

In [172]:
#пусть cutout будет 5 употреблений, т.е. всё, что употребляется 5 и более раз,
# считаем словом, по которому можно определить тон отзыва
positive_dictionary_filtered = dict(
    (word, number) for word, number in sorted_positive_dictionary.items() if number >= 5)
negative_dictionary_filtered = dict(
    (word, number) for word, number in sorted_negative_dictionary.items() if number >= 5)


In [173]:
# найдём слова, которые есть только в позитивных отзывах:
good_words = []
for word in positive_dictionary_filtered:
    if word not in negative_dictionary_filtered:
        good_words.append(word)
good_words

['best',
 'made',
 'performance',
 'masterpiece',
 'amazing',
 'movie',
 'well',
 'love',
 'spectacular',
 'cinematography',
 'scene']

In [174]:
# аналогично для негативных:
bad_words = []
for word in negative_dictionary_filtered:
    if word not in positive_dictionary_filtered:
        bad_words.append(word)
bad_words


['boring',
 'even',
 'waste',
 'character',
 'hour',
 'plot',
 'year',
 'terrible',
 'worst',
 'see']

In [175]:
# пусть для каждого отзыва в тесте будет считаться, сколько слов из "хорошего" списка
# там встречается, и сколько из "плохого". Если "хороших" слов больше или равно "плохих",
# считаем отзыв хорошим:

def decide(review):
    good_points = 0
    bad_points = 0
    for word in review.split():
        if word in good_words:
            good_points += 1
        if word in bad_words:
            bad_points += 1
    if good_points >= bad_points:
        return 1
    else: return 0


In [176]:
df_test["Prediction"] = df_test["Review"].apply(decide)
df_test

C:\Users\cheno\AppData\Local\Temp/ipykernel_15680/4257373135.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["Prediction"] = df_test["Review"].apply(decide)


,Rating,Review,Prediction
60,0,watched 30 minute dumb exaggerated poorly mad...,1
61,0,wow believe user score maybe missed something...,0
62,0,plot stupid none character memorable feel con...,0
63,0,one overrated movie ever technical level impr...,1
64,0,overlong boring dull much action enough sub...,0
65,1,awesome post apocalyptic first doubtful watch...,1
66,1,incredible relentless adrenaline fueled acti...,1
67,1,going honest seen know almost nothing one f...,1
68,1,back better ever really enjoy edge seat gre...,0
69,1,absolutely incredible found difficult blink n...,1


In [177]:
# посчитаем accuracy
TP = 4
FP = 2
TN = 3
FN = 1
accuracy = (TP + TN)/(TP + FP + TN + FN)
accuracy

0.7

In [178]:
#насчёт пункта 5 - вот как я предлагаю улучшить эту программу:
# 1) убрать стоп-слова и названия фильмов, а также слова movie и film, на этапе лемматизации (применила)
# 2) определять часть речи лемматизируемого слова для лучшего качества лемматизации nltk
# 3) заменять все эмоционально окрашенные слова на наиболее нейтральные синонимы и исходить уже из их частотности
